In [1]:
from collections import OrderedDict

In [2]:
class LoadSet(object):
    
    def __init__(self):
        self.loadlist = []
        self.names = set()
        
    def append(self,name,obj,load):
        name = name.lower()
        self.loadlist.append((name,obj,load))
        self.names.add(name)
        
    def iterloads(self,name):
        name = name.lower()
        ##if name not in self.names:
        ##    raise ValueError("Invalid load name: {}.  Must be one of: {}"
        ##                    .format(name,', '.join(sorted(list(self.names)))))
        for n,o,l in self.loadlist:
            if n == name:
                yield (o,l,1.0)
                
    def __len__(self):
        return len(self.names)
    
    def __iter__(self):
        return iter(self.loadlist)
    
    def __contains__(self,key):
        return key.lower() in self.names

In [3]:
##test:
ls = LoadSet()
for n,o,l in [('wind',10,20),('Wind',20,30),('live',30,40),('snow',40,50),('wind',50,60),('LIVE',60,70)]:
    ls.append(n,o,l)
ls.names

{'live', 'snow', 'wind'}

In [4]:
##test:
for t in ls:
    print(t)

('wind', 10, 20)
('wind', 20, 30)
('live', 30, 40)
('snow', 40, 50)
('wind', 50, 60)
('live', 60, 70)


In [5]:
##test:
'live' in ls, 'xxx' in ls

(True, False)

In [6]:
class LoadCombination(object):

    def __init__(self):
        self.combos = OrderedDict()
        
    def append(self,combo_name,load_name,factor):
        combo_name = combo_name.lower()
        load_name = load_name.lower()
        if combo_name not in self.combos:
            self.combos[combo_name] = OrderedDict()
        d = self.combos[combo_name]
        if load_name in d:
            raise ValueError("Load '{}' is multiply defined on combo '{}'".format(load_name.combo_name))
        d[load_name] = factor
        
    def iterloads(self,name,loadset):
        name = name.lower()
        if name not in self.combos:
            raise ValueError("Invalid load combination name: {}; must be one of '{}'"
                            .format(name,', '.join(sorted(self.combos.keys()))))
        for load_name,factor in self.combos[name].items():
            for obj,load,f in loadset.iterloads(load_name):
                yield obj,load,f*factor
                
    def __len__(self):
        return len(self.combos)
    
    def __iter__(self):
        for cname,llist in self.combos.items():
            for lname,factor in llist.items():
                yield cname,lname,factor
                
    def __contains__(self,key):
        return key.lower() in self.combos

In [7]:
##test:
lc = LoadCombination()
for c,l,f in [('Case-1','Wind',1.25),('Case-2','Wind',1.5),('Case-1','live',1.5),('case-2','live',2),('case-1','snow',3)]:
    lc.append(c,l,f)
lc.combos

OrderedDict([('case-1',
              OrderedDict([('wind', 1.25), ('live', 1.5), ('snow', 3)])),
             ('case-2', OrderedDict([('wind', 1.5), ('live', 2)]))])

In [8]:
##test:
for t in lc.iterloads('case-2',ls):
    print(t)

(10, 20, 1.5)
(20, 30, 1.5)
(50, 60, 1.5)
(30, 40, 2.0)
(60, 70, 2.0)


In [9]:
##test:
for t in lc:
    print(t)

('case-1', 'wind', 1.25)
('case-1', 'live', 1.5)
('case-1', 'snow', 3)
('case-2', 'wind', 1.5)
('case-2', 'live', 2)


In [10]:
##test:
'CASE-2' in lc, 'foo' in lc

(True, False)